In [ ]:
import os
import numpy as np
import soundfile as sf
from scipy.ndimage import gaussian_filter1d
import pandas as pd

def estimate_snr_weighted(audio_signal, base_sigma=1.2, min_segment_length=500, max_segment_length=1500, adaptive_factor=0.01):
    """
    Estimating the Signal-to-Noise Ratio (SNR) of a given speech signal using weighted adaptive smoothing,
    dynamic segment lengths, and signal power scaling for higher SNR.
    """
    if len(audio_signal.shape) > 1:
        audio_signal = np.mean(audio_signal, axis=1)

    normalized_signal = audio_signal / np.max(np.abs(audio_signal))
    power_signal = np.mean(normalized_signal ** 2)
    max_signal_value = np.max(np.abs(normalized_signal))
    adaptive_noise_threshold = adaptive_factor * max_signal_value

    smoothed_signal = np.zeros_like(normalized_signal)
    total_samples = len(normalized_signal)
    pos = 0

    while pos < total_samples:
        segment_length = np.random.randint(min_segment_length, max_segment_length)
        end = min(pos + segment_length, total_samples)
        segment = normalized_signal[pos:end]
        segment_power = np.abs(segment).mean()

        if segment_power < adaptive_noise_threshold:
            local_sigma = base_sigma + 3
        else:
            local_sigma = base_sigma

        smoothed_signal[pos:end] = gaussian_filter1d(segment, sigma=local_sigma)
        pos = end

    noise = normalized_signal - smoothed_signal
    power_noise = np.mean(noise ** 2)
    snr = 10 * np.log10(power_signal / power_noise)

    return snr

def calculate_snr_for_folder(folder_path, output_excel_file):
    """
    Calculates the SNR for all audio files in a given folder and saves the results in an Excel file.
    
    Args:
    folder_path (str): Path to the folder containing audio files.
    output_excel_file (str): Path to the output Excel file to save the results.
    """
    results = []
    audio_extensions = ('.wav', '.flac', '.ogg')  # Add other audio extensions if needed

    for filename in os.listdir(folder_path):
        if filename.endswith(audio_extensions):
            file_path = os.path.join(folder_path, filename)
            try:
                # Reading the audio file
                audio_signal, fs = sf.read(file_path)
                
                # Estimating SNR
                snr_estimated = estimate_snr_weighted(audio_signal)
                
                # Appending the result
                results.append({'Filename': filename, 'SNR (dB)': snr_estimated})
                print(f"Processed {filename}: SNR = {snr_estimated:.2f} dB")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    # Saving results to Excel file
    df = pd.DataFrame(results)
    df.to_excel(output_excel_file, index=False)
    print(f"Results saved to {output_excel_file}")

# Folder containing the audio files
folder_path = r'C:\Users\Lenovo\Desktop\New folder'


# Output Excel file path
output_excel_file = r'C:\Users\Lenovo\Desktop\New folder\snr_results.xlsx'  # The file where SNR results will be saved

# Calculating SNR for all files in the folder and saving to Excel
calculate_snr_for_folder(folder_path, output_excel_file)
